<a href="https://colab.research.google.com/github/gaveter/Project_cs/blob/master/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Project 2019-20
1. Normalize the loan_lenders table. In the normalized table, each row must have one loan_id and one lender.
2. For each loan, add a column duration corresponding to the number of days between the disburse time and the planned expiration time.
3. Find the lenders that have funded at least twice.
4. For each country, compute how many loans have involved that country as borrowers.
5. For each country, compute the overall amount of money borrowed.
6. Like the previous point, but expressed as a percentage of the overall amount lent.
7. Like the three previous points, but split for each year (with respect to disburse time).
8. For each lender, compute the overall amount of money lent. For each loan that has more than one lender, you must assume that all lenders contributed the same amount.
9. For each country, compute the difference between the overall amount of money lent and the overall amount of money borrowed.
10. Which country has the highest ratio between the difference computed at the previous point and the population?
11. Which country has the highest ratio between the difference computed at point 9 and the population that is not below the poverty line?
12. For each year, compute the total amount of loans. Each loan that has planned expiration time and disburse time in different years must 
have its amount distributed proportionally to the number of days in each year. For example, a loan with disburse time December 1st, 2016,
 planned expiration time January 30th 2018, and amount 5000USD has an amount of 5000USD * 31 / (31+365+30) = 363.85 for 2016, 5000USD * 365 / (31+365+30) = 4284.04
for 2017, and 5000USD * 30 / (31+365+30) = 352.11 for 2018.
© 2019 GitHub, Inc.

## Load libraries

In [0]:
import pandas as pd
from datetime import datetime
import re
import numpy as np

## Task1

### Load csv (loans_lenders)

In [4]:
loans_lender = pd.read_csv('/content/drive/My Drive/Foundation of computer science/additional-kiva-snapshot/loans_lenders.csv')
len(loans_lender)

1387432

In [5]:
loans_lender.head()

,loan_id,lenders
0,483693,"muc888, sam4326, camaran3922, lachheb1865, reb..."
1,483738,"muc888, nora3555, williammanashi, barbara5610,..."
2,485000,"muc888, terrystl, richardandsusan8352, sherri4..."
3,486087,"muc888, james5068, rudi5955, daniel9859, don92..."
4,534428,"muc888, niki3008, teresa9174, mike4896, david7..."


In [0]:
#loans_lender['lengthOriginal']=loans_lender.apply(lambda x: len(x['lenders']),axis=1)

In [0]:
loans_lender["lenders"]=loans_lender["lenders"].str.split(', ') #creazione lista per ogni riga
loans_lender_norm=loans_lender.explode('lenders')

In [8]:
loans_lender_norm.head(10)

,loan_id,lenders
0,483693,muc888
0,483693,sam4326
0,483693,camaran3922
0,483693,lachheb1865
0,483693,rebecca3499
0,483693,karlheinz4543
0,483693,jerrydb
0,483693,paula8951
0,483693,gmct
0,483693,amra9383


### Result

In [9]:
loans_lender_norm=loans_lender_norm.reset_index(drop=True)
loans_lender_norm.head()

,loan_id,lenders
0,483693,muc888
1,483693,sam4326
2,483693,camaran3922
3,483693,lachheb1865
4,483693,rebecca3499


## Task2
- For each loan, add a column duration corresponding to the number of days between the disburse time and the planned expiration time

### Load csv (loans)

In [0]:
loan = pd.read_csv('/content/drive/My Drive/Foundation of computer science/additional-kiva-snapshot/loans.csv')

In [12]:
loan.head()

,loan_id,loan_name,original_language,description,description_translated,funded_amount,loan_amount,status,activity_name,sector_name,loan_use,country_code,country_name,town_name,currency_policy,currency_exchange_coverage_rate,currency,partner_id,posted_time,planned_expiration_time,disburse_time,raised_time,lender_term,num_lenders_total,num_journal_entries,num_bulk_entries,tags,borrower_genders,borrower_pictured,repayment_interval,distribution_model
0,657307,Aivy,English,"Aivy, 21 years of age, is single and lives in ...",NaN,125.0,125.0,funded,General Store,Retail,to buy additional inventories for her sari-sa...,PH,Philippines,Ozamiz - Ozamiz City,shared,0.1,PHP,126.0,2014-01-15 02:23:45.000 +0000,2014-02-14 03:30:06.000 +0000,2013-12-22 08:00:00.000 +0000,2014-01-15 04:48:22.000 +0000,7.0,3,2,1,NaN,female,true,irregular,field_partner
1,657259,Idalia Marizza,Spanish,"Doña Idalia, esta casada, tiene 57 años de eda...","Idalia, 57, is married and lives with her husb...",400.0,400.0,funded,Used Clothing,Clothing,"To buy American clothing such as blouses, shir...",HN,Honduras,"La Lopez, Choloma",shared,0.1,HNL,201.0,2014-01-14 20:23:20.000 +0000,2014-03-26 22:25:07.000 +0000,2013-12-20 08:00:00.000 +0000,2014-02-25 06:42:06.000 +0000,8.0,11,2,1,NaN,female,true,monthly,field_partner
2,658010,Aasia,English,Aasia is a 45-year-old married lady and she ha...,NaN,400.0,400.0,funded,General Store,Retail,"to buy stock of rice, sugar and flour",PK,Pakistan,Lala Musa,shared,0.1,PKR,245.0,2014-01-16 11:32:58.000 +0000,2014-02-15 21:10:05.000 +0000,2014-01-09 08:00:00.000 +0000,2014-01-24 23:06:18.000 +0000,14.0,16,2,1,"#Woman Owned Biz, #Supporting Family, user_fav...",female,true,monthly,field_partner
3,659347,Gulmira,Russian,"Гулмире 36 лет, замужем, вместе с супругом вос...",Gulmira is 36 years old and married. She and ...,625.0,625.0,funded,Farming,Agriculture,"to buy cucumber and tomato seeds, as well as f...",KG,Kyrgyzstan,"Aravan village, Osh region",shared,0.1,KGS,171.0,2014-01-20 09:59:48.000 +0000,2014-02-21 03:10:02.000 +0000,2014-01-17 08:00:00.000 +0000,2014-01-22 05:29:28.000 +0000,14.0,21,2,1,user_favorite,female,true,monthly,field_partner
4,656933,Ricky\t,English,Ricky is a farmer who currently cultivates his...,NaN,425.0,425.0,funded,Farming,Agriculture,to buy organic fertilizer and agrochemical pr...,PH,Philippines,"Baleleng, Sto. Thomas, Isabela",shared,0.1,PHP,123.0,2014-01-14 05:46:21.000 +0000,2014-02-13 06:10:02.000 +0000,2013-12-17 08:00:00.000 +0000,2014-01-14 17:29:27.000 +0000,7.0,15,2,1,"#Animals, #Eco-friendly, #Sustainable Ag",male,true,bullet,field_partner


### Check missing values

In [0]:
missing_loan = pd.isnull(loan[["planned_expiration_time"]])

In [14]:
loan.loc[64:69][["planned_expiration_time","disburse_time"]]

,planned_expiration_time,disburse_time
64,NaN,2010-11-18 08:00:00.000 +0000
65,NaN,2010-11-25 08:00:00.000 +0000
66,NaN,2010-11-25 08:00:00.000 +0000
67,NaN,2010-11-16 08:00:00.000 +0000
68,2016-11-03 06:10:05.000 +0000,2016-09-07 07:00:00.000 +0000
69,NaN,2010-11-10 08:00:00.000 +0000


In [15]:
type(loan.loc[67][["planned_expiration_time"]][0]), type(loan.loc[68][["planned_expiration_time"]][0])

(float, str)

### Conversion attribute 'planned_expiration_time', 'disburse_time' to string

In [0]:
loan["planned_expiration_time"] = loan.apply(lambda x: str(x["planned_expiration_time"]),axis=1) #each row (value is float)
loan["disburse_time"] = loan.apply(lambda x: str(x["disburse_time"]),axis=1)

In [17]:
loan.loc[68]["disburse_time"]

'2016-09-07 07:00:00.000 +0000'

In [0]:
def difference_dates(date1,date2):
  if (date1=="nan" or date2=="nan"):
    return(np.nan) #float
  date1 = date1[:-10] #delete part +0000
  date1 = datetime.strptime(date1,"%Y-%m-%d %H:%M:%S")
  date2 = date2[:-10]
  date2 = datetime.strptime(date2,"%Y-%m-%d %H:%M:%S")
  diff = (date1 - date2).days
  return(diff)

In [0]:
loan["duration"] = loan.apply(lambda x: difference_dates(x["planned_expiration_time"],x["disburse_time"]),axis=1)

In [20]:
zeroDuration = loan[loan["duration"]==0][["planned_expiration_time","disburse_time","duration"]]
len(zeroDuration)

772

### Result

In [21]:
loan.head()[["planned_expiration_time","disburse_time","duration"]]

,planned_expiration_time,disburse_time,duration
0,2014-02-14 03:30:06.000 +0000,2013-12-22 08:00:00.000 +0000,53.0
1,2014-03-26 22:25:07.000 +0000,2013-12-20 08:00:00.000 +0000,96.0
2,2014-02-15 21:10:05.000 +0000,2014-01-09 08:00:00.000 +0000,37.0
3,2014-02-21 03:10:02.000 +0000,2014-01-17 08:00:00.000 +0000,34.0
4,2014-02-13 06:10:02.000 +0000,2013-12-17 08:00:00.000 +0000,57.0


## Task 3
- Find the lenders that have funded at least twice

In [22]:
loan['status'].unique()

array(['funded', 'expired', 'refunded', 'fundRaising'], dtype=object)

In [0]:
fundedLoan=loan[['loan_id','status']]

In [24]:
lenders_funded = pd.merge(loans_lender_norm,fundedLoan,how="inner",on="loan_id")
lenders_funded.head()

,loan_id,lenders,status
0,483693,muc888,funded
1,483693,sam4326,funded
2,483693,camaran3922,funded
3,483693,lachheb1865,funded
4,483693,rebecca3499,funded


In [0]:
numberfunded_lenders = lenders_funded.groupby('lenders')['loan_id'].count().reset_index(level=0)
numberfunded_lenders.columns=['lenders','count']

In [0]:
numberfunded_lenders = numberfunded_lenders[numberfunded_lenders['count']>=2]
numberfunded_lenders = numberfunded_lenders.sort_values('count',ascending=False)

### Result

In [27]:
numberfunded_lenders.head()

,lenders,count
462833,gooddogg1,177645
1324411,trolltech4460,150762
462243,gmct,128159
1008585,nms,104314
1290437,themissionbeltco,81434


## Task 4
- For each country, compute how many loans have involved that country as borrowers.

In [28]:
loan[["country_name","funded_amount","loan_amount","num_lenders_total"]]

,country_name,funded_amount,loan_amount,num_lenders_total
0,Philippines,125.0,125.0,3
1,Honduras,400.0,400.0,11
2,Pakistan,400.0,400.0,16
3,Kyrgyzstan,625.0,625.0,21
4,Philippines,425.0,425.0,15
...,...,...,...,...
1419602,Kenya,400.0,400.0,16
1419603,Philippines,300.0,300.0,12
1419604,Ghana,2425.0,2425.0,76
1419605,Philippines,100.0,100.0,3


In [29]:
loan['status'].unique()

array(['funded', 'expired', 'refunded', 'fundRaising'], dtype=object)

In [0]:
funded=loan[loan['status']=='funded'][['loan_id','country_name','status','currency','funded_amount','loan_amount','num_lenders_total']].head(2)
expired=loan[loan['status']=='expired'][['loan_id','country_name','status','funded_amount','loan_amount','num_lenders_total']].head(2)
refunded=loan[loan['status']=='refunded'][['loan_id','country_name','status','funded_amount','loan_amount','num_lenders_total']].head(2)
fundRaising=loan[loan['status']=='fundRaising'][['loan_id','country_name','status','funded_amount','loan_amount','num_lenders_total']].head(2)

In [31]:
esempio = pd.concat([funded,expired,refunded,fundRaising],axis=0)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [32]:
esempio

,country_name,currency,funded_amount,loan_amount,loan_id,num_lenders_total,status
0,Philippines,PHP,125.0,125.0,657307,3,funded
1,Honduras,HNL,400.0,400.0,657259,11,funded
22,Philippines,NaN,100.0,225.0,957461,4,expired
25,Philippines,NaN,225.0,300.0,959145,6,expired
361,Kenya,NaN,0.0,800.0,1158634,0,refunded
1420,Mexico,NaN,2075.0,2625.0,662566,67,refunded
5067,Ghana,NaN,0.0,225.0,1444070,0,fundRaising
16167,Lebanon,NaN,1375.0,2100.0,1431627,53,fundRaising


In [33]:
loan["status"].unique()

array(['funded', 'expired', 'refunded', 'fundRaising'], dtype=object)

### Check missing values

In [34]:
missing = pd.isnull(loan[['country_name']])
missing[missing['country_name']==True]

,country_name


In [0]:
country_numberLoan = loan.groupby('country_name')['status'].count()
country_numberLoan = country_numberLoan.sort_values(ascending=False)

### Result

In [36]:
country_numberLoan.head()

country_name
Philippines    285336
Kenya          143699
Peru            86000
Cambodia        79701
El Salvador     64037
Name: status, dtype: int64

## Task 5
- For each country, compute the overall amount of money borrowed.

In [0]:
country_amountLoan = loan.groupby('country_name')['funded_amount'].sum()
country_amountLoan = country_amountLoan.sort_values(ascending=False)

### Result

In [38]:
country_amountLoan.head()

country_name
Philippines    97045925.0
Peru           78702800.0
Kenya          63650255.0
Paraguay       53715200.0
Cambodia       50829425.0
Name: funded_amount, dtype: float64

## Task 6
- Like the previous point, but expressed as a percentage of the overall amount lent.

In [0]:
country_amountLoan_perc=pd.DataFrame(country_amountLoan).reset_index(level=0)
country_amountLoan_perc.columns=['country','total']

In [40]:
total_lent=country_amountLoan_perc['total'].sum()
total_lent

1130185170.0

In [0]:
country_amountLoan_perc['percentage']=round(country_amountLoan_perc['total']/total_lent*100,2)

### Result

In [42]:
country_amountLoan_perc.head()

,country,total,percentage
0,Philippines,97045925.0,8.59
1,Peru,78702800.0,6.96
2,Kenya,63650255.0,5.63
3,Paraguay,53715200.0,4.75
4,Cambodia,50829425.0,4.50


## Task 7
- Like the three previous points, but split for each year (with respect to disburse time).

In [0]:
loanYear = loan[['country_name','disburse_time','funded_amount','loan_amount']]

In [44]:
missing = pd.isnull(loanYear[['disburse_time']])
missing[missing['disburse_time']==True]

,disburse_time


In [45]:
len(loanYear[loanYear['disburse_time']=="nan"])

2813

In [0]:
loanYear = loanYear[loanYear['disburse_time']!='nan']

In [47]:
inizio=datetime.now()
print("Inizio: "+inizio.strftime("%H:%M:%S"))
loanYear['year']=loanYear.apply(lambda x: int(x["disburse_time"].split('-')[0]),axis=1)
fine=datetime.now()
print("Fine: "+fine.strftime("%H:%M:%S"))
diff= fine - inizio
diff

Inizio: 10:38:09
Fine: 10:38:30


datetime.timedelta(0, 21, 220019)

In [0]:
countryAmount_year=loanYear.groupby(['country_name','year'])['funded_amount'].sum()
countryAmount_year = pd.DataFrame(countryAmount_year)

### Result

In [53]:
countryAmount_year.head()

funded_amount
country_name year               
Afghanistan  2007       194975.0
             2008       365375.0
             2009       581825.0
             2010       563350.0
             2011       245125.0

## Task 8
- For each lender, compute the overall amount of money lent. For each loan that has more than one lender, you must assume that all lenders contributed the same amount.

In [54]:
numberLoans_lender_norm=loans_lender_norm.groupby('loan_id')['lenders'].count()
numberLoans_lender_norm=pd.DataFrame(numberLoans_lender_norm).reset_index(level=0)
numberLoans_lender_norm

,loan_id,lenders
0,84,3
1,85,2
2,86,3
3,88,3
4,89,4
...,...,...
1387427,1444051,1
1387428,1444053,1
1387429,1444058,1
1387430,1444063,1


In [0]:
loans_lender = loan[['loan_id','num_lenders_total','funded_amount']]

In [71]:
numberLoans = pd.merge(loans_lender,numberLoans_lender_norm,how="inner",on="loan_id")
numberLoans.head()

,loan_id,num_lenders_total,funded_amount,lenders
0,657307,3,125.0,3
1,657259,11,400.0,7
2,658010,16,400.0,14
3,659347,21,625.0,17
4,656933,15,425.0,14


In [0]:
correct = numberLoans[numberLoans['num_lenders_total']==numberLoans['lenders']]
wrong = numberLoans[numberLoans['num_lenders_total']!=numberLoans['lenders']]

In [70]:
wrong.head()

,loan_id,num_lenders_total,funded_amount,lenders
1,657259,11,400.0,7
2,658010,16,400.0,14
3,659347,21,625.0,17
4,656933,15,425.0,14
5,659605,13,350.0,14


In [66]:
#plot barchart da fare
print("Info giuste: "+str(len(correct)/len(numberLoans)))
print("Info sbagliate: "+str(len(wrong)/len(numberLoans)))

Info giuste: 0.1917209397532701
Info sbagliate: 0.8082790602467299


In [0]:
loans_lender_correct = pd.merge(loans_lender,numberLoans_lender_norm,how='inner',on='loan_id')
loans_lender_correct.drop(columns='num_lenders_total',inplace=True)

In [0]:
loans_lender_correct['amount']=round(loans_lender_correct['funded_amount']/loans_lender_correct['lenders'],2)

In [83]:
loans_lender_correct.head()

,loan_id,funded_amount,lenders,amount
0,657307,125.0,3,41.67
1,657259,400.0,7,57.14
2,658010,400.0,14,28.57
3,659347,625.0,17,36.76
4,656933,425.0,14,30.36


In [0]:
amount_loan_lender = pd.merge(loans_lender_norm,loans_lender_correct,how="inner",on='loan_id')
amount_loan_lender.drop(columns=['funded_amount','lenders_y'],inplace=True)
amount_loan_lender.columns=['loan_id','lenders','amount'] 

In [0]:
amountLender = amount_loan_lender.groupby('lenders')['amount'].sum()
amountLender = pd.DataFrame(amountLender).reset_index(level=0)

In [0]:
amountLender = amountLender.sort_values(by='amount',ascending=False)

In [0]:
amountLender['amountFormat']=amountLender.apply(lambda x: format(x['amount'],'f'),axis=1)

In [116]:
amountLender.head()

,lenders,amount,amountFormat
462833,gooddogg1,8.140636e+06,8140636.400000
1324411,trolltech4460,5.634790e+06,5634789.740000
1008585,nms,4.453638e+06,4453638.040000
462243,gmct,4.445122e+06,4445122.169999
158450,bob4739,4.439756e+06,4439755.910000
